# Replicate ICs from McNally *et. al.* (2012)

In [ ]:
DATA_DIR = '/home/pal.balazs/data/kelvin_helmholtz/ICs/'

In [ ]:
import os
import h5py

import numpy as np
from scipy.interpolate import griddata

import matplotlib.pyplot as plt

In [ ]:
def get_keys(s):
    with h5py.File(os.path.join(DATA_DIR, s), "r") as f:
        keys = list(f['PartType0'].keys())
    return keys

In [ ]:
def get_data(s):
    values = {}
    with h5py.File(os.path.join(DATA_DIR, s), "r") as f:
        for k in get_keys(s):
            values[k] = f['PartType0'][k][:]
    return values

In [ ]:
def get_grid_data(s, N):
    X, Y, D, _ = get_data(s)
    xi, yi = np.meshgrid(np.linspace(0, 1, N), np.linspace(0, 1, N),
                         indexing='ij')
    return griddata(np.stack((X, Y), axis=1), D, (xi, yi), method='cubic')

## Generate initial conditions with smooth boundaries

### Hyperparameters

In [ ]:
N  = 192   # Number of particles in a single dimension
Lx = 1.0    # Size of the box along the x-axis
Ly = 1.0    # Size of the box along the y-axis
dh = 0.025  # Smoothing length of V and D on the fluid boundary

In [ ]:
KH = [0.25, 0.75]

### Determine smoothing length

In [ ]:
delta_x = (Lx**2 / (N*Lx)**2)**0.5
delta_y = (Ly**2 / (N*Ly)**2)**0.5
delta_x, delta_y

### Generate coordinates

In [ ]:
# Coordinates on a grid
xi   = np.linspace(0, Lx, int(N*Lx), endpoint=False)
yi   = np.linspace(0, Ly, int(N*Ly), endpoint=False)
X, Y = np.meshgrid(xi, yi, sparse=False, indexing='xy')
X, Y = X.flatten(), Y.flatten()
Z    = np.zeros(X.size)

In [ ]:
F = (Y - np.min(Y)) / np.max(Y - np.min(Y))

### Generate densities

In [ ]:
rho_1, rho_2 = 1, 2  # Densities of the two interacting fluids
rho_m = (rho_1 - rho_2) * 0.5

In [ ]:
D = np.zeros(X.size)

#
mask    = (F >= 0) & (F < KH[0])
D[mask] = rho_1 - rho_m * np.exp((F[mask] - KH[0]) / dh)

#
mask    = (F >= KH[0]) & (F < 0.5)
D[mask] = rho_2 + rho_m * np.exp((-F[mask] + KH[0]) / dh)

#
mask    = (F >= 0.5) & (F < KH[1])
D[mask] = rho_2 + rho_m * np.exp((-(KH[1] - F[mask])) / dh)

#
mask    = (F >= KH[1]) & (F <= 1.0)
D[mask] = rho_1 - rho_m * np.exp((-(F[mask] - KH[1])) / dh)

In [ ]:
nr, nc = 1, 2
fig, axes = plt.subplots(nr, nc, figsize=(nc*6, nr*5))
axes[0].plot(D)
axes[1].plot(sorted(D))
plt.show()

### Generate velocities

In [ ]:
u_1, u_2 = 0.5, -0.5  # Velocities of the two interacting fluids
u_m = (u_1 - u_2) * 0.5

In [ ]:
vx = np.zeros(X.size)

#
mask    = (F >= 0) & (F < KH[0])
vx[mask] = u_1 - u_m * np.exp((F[mask] - KH[0]) / dh)

#
mask    = (F >= KH[0]) & (F < 0.5)
vx[mask] = u_2 + u_m * np.exp((-F[mask] + KH[0]) / dh)

#
mask    = (F >= 0.5) & (F < KH[1])
vx[mask] = u_2 + u_m * np.exp((-(KH[1] - F[mask])) / dh)

#
mask    = (F >= KH[1]) & (F <= 1.0)
vx[mask] = u_1 - u_m * np.exp((-(F[mask] - KH[1])) / dh)

In [ ]:
vy = 0.01 * np.sin(4.0 * np.pi * X)

In [ ]:
vz = np.zeros(X.size)

In [ ]:
nr, nc = 1, 3
fig, axes = plt.subplots(nr, nc, figsize=(nc*6, nr*5))
axes[0].plot(Y, vx)
axes[1].scatter(X, vy)
axes[2].plot(vz)
plt.show()

### Calculate internal energy

In [ ]:
P = 5./2.      # Constant pressure as seen in Springel (2010) and McNally (2012)
gamma = 5./3.  # Polytropic/adiabatic index of ideal EoS

In [ ]:
E = P / ((gamma - 1) * D)

### Calculate masses

In [ ]:
M = D / D.size

## Inspect generated IC

In [ ]:
nr, nc = 2, 2
fig, axes = plt.subplots(nr, nc, figsize=(nc*6*Lx, nr*6*Ly), dpi=120,
                         facecolor='black')
fig.subplots_adjust(wspace=0.01, hspace=0.01)
axes = axes.flatten()

for ax in axes:
    ax.axis(False)
    ax.set_aspect('equal')
    ax.set_xlim(np.min(X), np.max(X))
    ax.set_ylim(np.min(Y), np.max(Y))

# X-velocities
ax = axes[0]
ax.scatter(X, Y, c=vx, cmap='viridis', s=0.1)

# Density
ax = axes[1]
ax.scatter(X, Y, c=D, cmap='viridis', s=0.1)

# Internal energy
ax = axes[2]
ax.scatter(X, Y, c=E, cmap='viridis', s=0.1)

# Masses
ax = axes[3]
ax.scatter(X, Y, c=M, cmap='viridis', s=0.1)

plt.show()

## Save IC

In [ ]:
file = h5py.File(os.path.join(DATA_DIR, f'kh_mcnally_{N}.hdf5'), 'w')

h = file.create_group('Header')
NPART = np.array([X.size, 0, 0, 0, 0, 0])
h.attrs['NumPart_ThisFile'] = NPART
h.attrs['NumPart_Total'] = NPART
h.attrs['NumPart_Total_HighWord'] = 0*NPART
h.attrs['MassTable'] = np.zeros(6)
h.attrs['Time'] = 0.0
h.attrs['NumFilesPerSnapshot'] = 1
h.attrs['Flag_DoublePrecision'] = 0

p = file.create_group('PartType0')
p.create_dataset('Coordinates', data=np.column_stack((X, Y, Z)))
p.create_dataset('Velocities', data=np.column_stack((vx, vy, vz)))
p.create_dataset('ParticleIDs', data=np.arange(1, N**2+1, 1))
#p.create_dataset('ParticleChildIDsNumber', data=np.zeros(N**2))
#p.create_dataset('ParticleIDGenerationNumber', data=np.zeros(N**2))
p.create_dataset('Masses', data=M)
p.create_dataset('Density', data=D)
p.create_dataset('InternalEnergy', data=E)
p.create_dataset('SmoothingLength', data=((M / D)**(1/2)))

file.close()

In [ ]:
s = os.path.join(DATA_DIR, 'test_kh_ics.hdf5')

In [ ]:
values = get_data(s)

In [ ]:
values